In [3]:
import tensorflow as tf
import pandas as pd

from models.FCN import FCN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from utils.helper import *
from utils.constants import *

df = read_dataset("ArrowHead")

X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)
# apply one-hot encoder
y_train=to_categorical(df[1])
y_test=to_categorical(df[3])

model = FCN(input_size=X_train.shape[1:] , nb_classes=y_train.shape[1], filters=[128,256,128])
model.compile(optimizer=Adam(learning_rate=0.01), loss=CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

test (36, 251, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 251, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 251, 128)          1152      
                                                                 
 batch_normalization (Batch  (None, 251, 128)          512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 251, 128)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 251, 256)          164096    
                                                                 
 batch_normalization_1 (Bat  (None, 251, 256)          1024      
 chNormalization)                          

2024-02-16 15:31:46.514831: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 15:31:46.552418: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 15:31:46.552685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
import tensorflow as tf
import pandas as pd

from models.FCN import FCN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from utils.helper import *
from utils.constants import *



results=[]
histories=[]

try:
    for dataset in  UNIVARIATE_DATASET_NAMES_2018:
        with tf.device("/device:GPU:0"):
            # Load data
            df = read_dataset(dataset)

            X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
            X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)

            # apply one-hot encoder
            y_train=to_categorical(df[1])
            y_test=to_categorical(df[3])
            print("test",X_train.shape)

            # Compile the model
            model = FCN(input_size=X_train.shape[1:] , nb_classes=y_train.shape[1], filters=[128,256,128])
            model.compile(optimizer=Adam(learning_rate=0.01), loss=CategoricalCrossentropy(), metrics=['accuracy'])

            callbacks =[
                tf.keras.callbacks.ModelCheckpoint(
                    f"best_models/best_model_distiller_{dataset}.tf", save_weights_only=True, monitor="lr"
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor="lr", factor=0.5, patience=50, min_lr=0.0001
                ),
            ]

            # Train the model
            history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data= (df[2], y_test), callbacks = callbacks)

            # Evaluate the model
            train_loss, train_accuracy = model.evaluate(X_train, y_train)
            test_loss, test_accuracy = model.evaluate(X_test, y_test)

            results.append([dataset, train_loss, train_accuracy, test_loss, test_accuracy])
            histories.append(history.history)
        # clear_gpu_memory()
            
    pd.DataFrame(results, columns=["Dataset","Train loss", "Train accuracy","Test loss", "Test accuracy"]).to_csv("result.csv", index=False)

except Exception as e:
    print("An error occurred:", e)


for i in range(len(histories)):
    plot_loss(histories[i], UNIVARIATE_DATASET_NAMES_2018[i])

test (36, 251, 1)
salam
Epoch 1/1000


2024-02-16 15:33:53.294738: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-16 15:33:54.629141: I external/local_xla/xla/service/service.cc:168] XLA service 0x560a7a872cc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-16 15:33:54.629163: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P400, Compute Capability 6.1
2024-02-16 15:33:54.634276: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708094034.710756  112611 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 [==============================] - 7s 1s/step - loss: 1.2106 - accuracy: 0.3056 - val_loss: 2.0402 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 2/1000
3/3 [==============================] - 0s 125ms/step - loss: 1.0299 - accuracy: 0.4722 - val_loss: 3.2486 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 3/1000
3/3 [==============================] - 0s 135ms/step - loss: 0.9827 - accuracy: 0.5556 - val_loss: 6.1188 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 4/1000
3/3 [==============================] - 0s 120ms/step - loss: 0.9651 - accuracy: 0.5278 - val_loss: 9.4277 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 5/1000
3/3 [==============================] - 0s 126ms/step - loss: 0.9694 - accuracy: 0.5556 - val_loss: 10.9245 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 6/1000
3/3 [==============================] - 0s 116ms/step - loss: 0.9513 - accuracy: 0.5556 - val_loss: 8.7621 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 7/1000
3/3 [==============================] - 0s 116ms/step - loss: 0.9097 - ac

2024-02-16 18:46:46.682484: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.33GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-16 18:46:46.756215: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.33GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-16 18:46:46.850623: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-16 18:46:46.970135: W external/local_tsl/tsl/framework/bfc_

1/2 [==============>...............] - ETA: 3s - loss: 1.4441 - accuracy: 0.0000e+00

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [10]:
results = pd.read_csv("result.csv")
results

,Dataset,Train loss,Train accuracy,Test loss,Test accuracy
0,ArrowHead,2.169613e-02,1.000000,0.588603,0.811429
1,Wine,5.724999e-01,0.701754,0.568843,0.722222
2,OliveOil,1.008327e+00,0.500000,1.068600,0.433333
3,Car,9.128553e-03,1.000000,0.179467,0.916667
4,BeetleFly,1.187496e-02,1.000000,0.521813,0.750000
5,Yoga,1.083056e-01,0.980000,0.460021,0.812667
6,InlineSkate,1.254727e+00,0.510000,1.976222,0.283636
7,FaceAll,3.761476e-07,1.000000,0.845623,0.850296
8,Ham,6.499323e-03,1.000000,1.159491,0.752381
9,MoteStrain,1.450405e-03,1.000000,0.249668,0.924121


In [11]:
df = pd.read_csv("results-uea-avg-std.csv")

# remove std
def remove_parenthesis(value):
    return float(value.split("(")[0])

def convert_to_percent(value):
    return float("%.2f" % float(value*100))

df.iloc[:, 1:] = df.iloc[:, 1:].map(remove_parenthesis)
results["Train accuracy"]=results["Train accuracy"].apply(convert_to_percent)
results["Test accuracy"]=results["Test accuracy"].apply(convert_to_percent)

df['Unnamed: 0'] = df['Unnamed: 0'].str.lower()
dataset_names = pd.Series(UNIVARIATE_DATASET_NAMES_2018, name="Unnamed: 0").str.lower()
df_copy = df.merge(dataset_names, on='Unnamed: 0', how='right')
df_copy.dropna(axis=0, inplace=True)

results['Unnamed: 0'] = results['Dataset'].str.lower()

# merge and remove unused columns
result = results.merge(df_copy[["Unnamed: 0","FCN"]],on="Unnamed: 0",how="right").drop("Unnamed: 0",axis=1)
result

,Dataset,Train loss,Train accuracy,Test loss,Test accuracy,FCN
0,ArrowHead,2.169613e-02,100.00,0.588603,81.14,84.3
1,Wine,5.724999e-01,70.18,0.568843,72.22,58.7
2,OliveOil,1.008327e+00,50.00,1.068600,43.33,72.3
3,Car,9.128553e-03,100.00,0.179467,91.67,90.5
4,BeetleFly,1.187496e-02,100.00,0.521813,75.00,86.0
5,Yoga,1.083056e-01,98.00,0.460021,81.27,83.9
6,InlineSkate,1.254727e+00,51.00,1.976222,28.36,33.9
7,FaceAll,3.761476e-07,100.00,0.845623,85.03,94.5
8,Ham,6.499323e-03,100.00,1.159491,75.24,71.8
9,MoteStrain,1.450405e-03,100.00,0.249668,92.41,93.7


In [11]:
plot_1v1_perf(result,"MLP","Test accuracy")